**Computes road usage using map matching technology (nrel/mappymatch)**

Map matching is a computation intensive task, and can give imprecise results when GPS data wasn't recorded at high enough frequency.

**Input**: "sources/data.csv" csv using the following columns:
* `transportation_mode`: used mode of transport for the trip, only passenger_car (1) and bike_trip (8) are considered here
* `start_time`: datetime of begining of trip
* `end_time`: datetime of end of trip
* `trace_gps`: list of (lon, lat) tuples
* `longitude`, `latitude`: coords of starting point (also included as first point of `trace_gps`)

**Input**: "sources/paris.geojson" Shape representation of Paris, to filter results to the inner city and improve performance

**Output**: geoJSON files "../static/data/car.geojson" and "../static/data/bike.geojson containing road shapes with the following metadata:
* `road_id`: An identifier for the road, expressed as two points
* `count`: Total number of trips on the road segment
* `stroke`: A color representation of `count`, from green to red

For anonymity, there must be at least 5 trips in a given road for data to be considered


In [ ]:
import pandas as pd
from mappymatch.constructs.trace import Trace
from mappymatch.utils.plot import plot_trace
from mappymatch.constructs.geofence import Geofence
from mappymatch.maps.nx.nx_map import NxMap, NetworkType
from mappymatch.utils.plot import plot_map
from mappymatch.matchers.lcss.lcss import LCSSMatcher
from mappymatch.utils.plot import plot_matches
from mappymatch.utils.plot import plot_path
from mappymatch.utils.crs import LATLON_CRS, XY_CRS
from shapely.geometry import Point
import geopandas as gpd
import folium
import json


In [ ]:
geofence = Geofence.from_geojson("sources/paris.geojson")

In [ ]:
nx_map = NxMap.from_geofence(geofence, network_type=NetworkType.DRIVE)

In [ ]:
#plot_map(nx_map)

In [ ]:
matcher = LCSSMatcher(nx_map)

In [ ]:
df = pd.read_csv("sources/data.csv")

In [ ]:
#df_in_paris = df[df.apply(lambda x: Point(x.longitude, x.latitude).within(geofence.geometry), axis=1)] 
df_in_paris = df

In [ ]:
df_in_paris_car = df_in_paris[df_in_paris['transportation_mode'] == 1] # 1 = passenger car
df_in_paris_cycle = df_in_paris[df_in_paris['transportation_mode'] == 8] # 8 = bike trip

In [ ]:
%%time
def gps_to_trace(gps):
    t = json.loads(gps)
    ddf = pd.DataFrame(t, columns=['lon','lat'])
    ddf = ddf[ddf.apply(lambda x: Point(x.lon, x.lat).within(geofence.geometry), axis=1)]
    if ddf.shape[0] == 0:
        return None
    trace = Trace.from_dataframe(ddf, lat_column='lat', lon_column='lon')
    return trace
df_in_paris_car['trace'] = df_in_paris_car['trace_gps'].apply(gps_to_trace)
df_in_paris_cycle['trace'] = df_in_paris_cycle['trace_gps'].apply(gps_to_trace)

In [ ]:
df_in_paris_car = df_in_paris_car[df_in_paris_car['trace'].notnull()]
df_in_paris_cycle = df_in_paris_cycle[df_in_paris_cycle['trace'].notnull()]

In [ ]:
#ddf = pd.DataFrame(df.iloc[11012]['trace_gps'], columns=['lon','lat'])
#trace = Trace.from_dataframe(ddf, lat_column='lat', lon_column='lon')
#df.iloc[11006]
m = plot_trace(df_in_paris_car.iloc[0]['trace'])
c = 0
for t in df_in_paris_car['trace']:
    c += 1
    if c > 10:
        break
    plot_trace(t, m=m)
m

In [ ]:
%%time

#trace = df.iloc[11007]['trace']
#df_in_paris_sample = df_in_paris#.sample(n=100)
match_results_car = matcher.match_trace_batch(df_in_paris_car['trace'], processes=4)

In [ ]:
%%time

match_results_cycle = matcher.match_trace_batch(df_in_paris_cycle['trace'], processes=4)

In [ ]:
import folium
start_lat = 48.8915079
start_long = 2.3495425
curr_map = folium.Map(location = [start_lat, start_long], zoom_start = 17)
for i in range(0, 10):
    try:
        plot_trace(df_in_paris_car.iloc[i]['trace'], point_color="blue", m=plot_path(match_results_car[i].path, crs=df_in_paris_car.iloc[i]['trace'].crs, m=curr_map))
    except:
        pass
curr_map

In [ ]:
def combine(match_results):
    df_combined = match_results[0].path_to_dataframe()
    for match in match_results[1:]:
        df_combined = pd.concat([df_combined, match.path_to_dataframe()], ignore_index=True)
    road_counts = df_combined['road_id'].value_counts().reset_index()
    road_counts.columns = ['road_id', 'count']
    df_combined = df_combined.merge(road_counts, on='road_id')
    return df_combined

df_combined_car = combine(match_results_car)
df_combined_cycle = combine(match_results_cycle)


In [ ]:
df_combined_car_filter = df_combined_car[df_combined_car['count']>4]
gdf_car = gpd.GeoDataFrame(df_combined_car_filter, geometry=df_combined_car_filter.geom, crs=XY_CRS)
gdf_car = gdf_car.to_crs(LATLON_CRS)

df_combined_cycle_filter = df_combined_cycle[df_combined_cycle['count']>4]
gdf_cycle = gpd.GeoDataFrame(df_combined_cycle_filter, geometry=df_combined_cycle_filter.geom, crs=XY_CRS)
gdf_cycle = gdf_cycle.to_crs(LATLON_CRS)

In [ ]:
# Export as geojson
import branca.colormap as cm

def convert_geojson(gdf, file):
    # cleanup
    gdf["road_id"] = gdf["road_id"].apply(str)
    gdf["geom"] = None

    # Colorize
    colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=0, vmax=gdf["count"].max())

    gdf["stroke"] = gdf["count"].apply(lambda x: colormap(x)[:-2])

    # Export
    gdf.to_file(file, driver="GeoJSON")

convert_geojson(gdf_car, "../static/data/car.geojson")
convert_geojson(gdf_cycle, "../static/data/cycle.geojson")

In [ ]:
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
# other mapping code (e.g. lines, markers etc.)
folium.GeoJson(gdf_car, style_function=lambda f: {"color": f['properties']['stroke']}).add_to(m)
folium.GeoJson(gdf_cycle, style_function=lambda f: {"color": f['properties']['stroke']}).add_to(m)
folium.LayerControl().add_to(m)
m